# Overview to approach "machine-readable" files
1. Try to load it in duckdb
2. Try to load it in pandas
3. Use command-line tools to determine the encoding
4. Re-encode the file
5. Repeat to try to load it in duckdb and pandas
6. Remove any escape characters using command-line tools

## Command-line dependencies
These can be installed using homebrew on Mac OS X:
```
brew install libiconv
brew install uchardet
```

In [1]:
# Load duckdb, which lets us efficiently load large files
import duckdb

# Load pandas, which lets us manipulate dataframes
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

# Set configrations on jupysql to directly output data to Pandas and to simplify the output that is printed to the notebook.
%config SqlMagic.autopandas = True

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Allow named parameters (python variables) in SQL cells
%config SqlMagic.named_parameters=True

# Connect jupysql to DuckDB using a SQLAlchemy-style connection string. Either connect to an in memory DuckDB, or a file backed db.
%sql duckdb:///:memory:

In [2]:
!wget https://www.lvhn.org/sites/default/files/2022-12/231689692_Lehigh_Valley_Hospital_StandardCharges.zip -P /tmp

--2023-08-31 10:14:43--  https://www.lvhn.org/sites/default/files/2022-12/231689692_Lehigh_Valley_Hospital_StandardCharges.zip
Resolving www.lvhn.org (www.lvhn.org)... 2620:12a:8001::1, 2620:12a:8000::1, 23.185.0.1
Connecting to www.lvhn.org (www.lvhn.org)|2620:12a:8001::1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37214088 (35M) [application/zip]
Saving to: ‘/tmp/231689692_Lehigh_Valley_Hospital_StandardCharges.zip’

231689692_Lehigh_Va 100%[===================>]  35.49M  6.77MB/s    in 5.2s    

2023-08-31 10:14:49 (6.85 MB/s) - ‘/tmp/231689692_Lehigh_Valley_Hospital_StandardCharges.zip’ saved [37214088/37214088]



In [5]:
!unzip /tmp/231689692_Lehigh_Valley_Hospital_StandardCharges.zip && mv 231689692_Lehigh_Valley_Hospital_StandardCharges.JSON ~/data/payless_health

Archive:  /tmp/231689692_Lehigh_Valley_Hospital_StandardCharges.zip
  inflating: 231689692_Lehigh_Valley_Hospital_StandardCharges.JSON  


In [6]:
ls -lh ~/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON

-rw-r--r--  1 me  staff   858M Dec  2  2022 /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON


In [7]:
!head /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON

{
"Standard Charges":[
{
"Header":"Lehigh Valley Hospital-Cedar Crest"
}
,
{
"Header":"Lehigh Valley Hospital-Muhlenberg"
}
,


In [5]:
file_path = '/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON'

In [6]:
%%sql
SELECT * FROM read_json_auto(:file_path, records=true, maximum_object_size=500000000)

RuntimeError: (duckdb.InvalidInputException) Invalid Input Error: Attempting to execute an unsuccessful or closed pending query result
Error: Invalid Input Error: Malformed JSON in file "/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON", at byte 1964 in record/value 2: invalid UTF-8 encoding in string. 
[SQL: SELECT * FROM read_json_auto(?, records=true, maximum_object_size=500000000)]
[parameters: ('/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON',)]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [7]:
!uchardet /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON

ISO-8859-2


In [8]:
import io
import chardet
with open(file_path, 'rb') as f:
    result = chardet.detect(f.read(5000000))  # or readline if the file is large

result

{'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}

In [10]:
!file -bI /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON

text/plain; charset=iso-8859-1


In [11]:
!iconv -f ISO-8859-1 -t UTF-8 /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON > /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON

In [12]:
file_path_utf8 = '/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON'

In [13]:
%%sql
SELECT * FROM read_json_auto(:file_path_utf8, records=true, maximum_object_size=500000000)

RuntimeError: (duckdb.InvalidInputException) Invalid Input Error: Attempting to execute an unsuccessful or closed pending query result
Error: Invalid Input Error: Malformed JSON in file "/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON", at byte 878701 in record/value 2: invalid escaped character in string. 
[SQL: SELECT * FROM read_json_auto(?, records=true, maximum_object_size=500000000)]
[parameters: ('/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON',)]
(Background on this error at: https://sqlalche.me/e/20/f405)
If you need help solving this issue, send us a message: https://ploomber.io/community


In [18]:
!od -j 878699 -N 16 -a /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON 

3264153    \   F   I   N   G   E   R  sp   L   S   N   ;  sp   S   U   B
3264173


The offending escape character seems to be a backslash. We can remove it using sed:
```
sed -i '' 's/\\//g' 230831-lehigh-valley.csv
```

Or with tr:
```
tr -d '\\' < 230831-lehigh-valley.csv > 230831-lehigh-valley.csv
```

In [19]:
!tr -d '\\' < /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.JSON > /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON

In [2]:
%%sql
SELECT * FROM read_json_auto('/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON', records=true, maximum_object_size=500000000)

,Standard Charges,Minimum & Maximum Allowed,NCC 4,NCC 5,NCC 5 RCH,NCC 6,NCC 9,NCC 14,NCC 14 RCH,NCC 15,...,NCC 484,NCC 485,NCC 486,NCC 494,NCC 499,NCC 526,NCC 535,NCC 536,NCC 571,NCC 574
0,[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,"[{'Payment Type': 'APR DRG', 'Description': 'L...","[{'Payment Type': 'MS DRG', 'Description': 'HE...",[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,[{'Header': 'Lehigh Valley Hospital-Reilly Chi...,[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...",[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,[{'Header': 'Lehigh Valley Hospital-Reilly Chi...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...",...,[{'Header': 'Lehigh Valley Hospital-Hecktown O...,[{'Header': 'Lehigh Valley Hospital-Hecktown O...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'APR DRG', 'Description': 'L...","[{'Payment Type': 'PER DIEM', 'Description': '...","[{'Header': 'Lehigh Valley Hospital-Carbon', '...",[{'Header': 'Lehigh Valley Hospital-Coordinate...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'APR DRG', 'Description': 'L..."


In [4]:
!head -n 100 /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON

{
"Standard Charges":[
{
"Header":"Lehigh Valley Hospital-Cedar Crest"
}
,
{
"Header":"Lehigh Valley Hospital-Muhlenberg"
}
,
{
"Header":"Lehigh Valley Hospital-Hecktown Oaks"
}
,
{
"Header":"Lehigh Valley Hospital-Reilly Children's Hospital"
}
,
{
"Header":"Lehigh Valley Hospital-Tilghman Surgery Center"
}
,
{
"Header":"Lehigh Valley Hospital-17th Street"
}
,
{
"Header":"Lehigh Valley Hospital-Carbon"
}
,
{
"Header":"Lehigh Valley Hospital-Coordinated Health Allentown"
}
,
{
"Header":"Lehigh Valley Hospital-Coordinated Health Bethlehem"
}
,
{
"Header":"Comprehensive Machine Readable File"
}
,
{
"Header":"Date of last Update: 12/15/2022"
}
,
{
"DESCRIPTION":"HB MED SURG PRIVATE BED"
,
"BILLING_CODE":"110"
,
"GROSS_CHARGES_IP":"$3,730.00"
,
"GROSS_CHARGES_OP":"N/A"
,
"SELF-PAY_PRICE_IP":"$1,865.00"
,
"SELF-PAY_PRICE_OP":"N/A"
}
,
{
"DESCRIPTION":"HB EBOLA PRIVATE"
,
"BILLING_CODE":"110"
,
"GROSS_CHARGES_IP":"$34,485.00"
,
"GROSS_CHARGES_OP":"N/A"
,
"SELF-PAY_PRICE_IP":"$17,242.50"
,
"SE

In [15]:
file_path_iso_8859_1_utf8 = '/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON.ISO-8859-1.utf8'

In [2]:
%%sql
SELECT * FROM read_json_auto('/Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.JSON.ISO-8859-1.utf8', records=true, maximum_object_size=500000000)

,Standard Charges,Minimum & Maximum Allowed,NCC 4,NCC 5,NCC 5 RCH,NCC 6,NCC 9,NCC 14,NCC 14 RCH,NCC 15,...,NCC 484,NCC 485,NCC 486,NCC 494,NCC 499,NCC 526,NCC 535,NCC 536,NCC 571,NCC 574
0,[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,"[{'Payment Type': 'APR DRG', 'Description': 'L...","[{'Payment Type': 'MS DRG', 'Description': 'HE...",[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,[{'Header': 'Lehigh Valley Hospital-Reilly Chi...,[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...",[{'Header': 'Lehigh Valley Hospital-Cedar Cres...,[{'Header': 'Lehigh Valley Hospital-Reilly Chi...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...",...,[{'Header': 'Lehigh Valley Hospital-Hecktown O...,[{'Header': 'Lehigh Valley Hospital-Hecktown O...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'APR DRG', 'Description': 'L...","[{'Payment Type': 'PER DIEM', 'Description': '...","[{'Header': 'Lehigh Valley Hospital-Carbon', '...",[{'Header': 'Lehigh Valley Hospital-Coordinate...,"[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'MS DRG', 'Description': 'HE...","[{'Payment Type': 'APR DRG', 'Description': 'L..."


# Parsing this json with `jq` 

1. Install `jq` with homebrew
2. Test it out with `jq . 230831-lehigh-valley.json`
3. Use `jq` to extract the `data` array: `jq .data 230831-lehigh-valley.json`
4. Use `jq` to extract the `data` array and write it to a file: `jq .data 230831-lehigh-valley.json > 230831-lehigh-valley.json`
5. Use `jq.py` (https://github.com/mwilliamson/jq.py) in python to process the 83 columns.

## Copilot prompt

`jq` command to retrieve column names from the json file:
````
jq .data[0] 230831-lehigh-valley.json
````

## Claude prompt
````
please write a jq command to retrieve the 83 columns names from the json file at /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON where the column names are at the top level and followed by lists.
````

Claude response:
```
jq -r 'keys | .[]' /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON   
```




In [15]:
!jq -r 'keys | .[]' /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON   

Minimum & Maximum Allowed
NCC 100
NCC 102
NCC 105
NCC 108
NCC 117
NCC 128
NCC 133
NCC 14
NCC 14 RCH
NCC 148
NCC 15
NCC 151
NCC 153
NCC 157
NCC 171
NCC 19
NCC 26
NCC 265
NCC 269
NCC 269 RCH
NCC 270
NCC 270 RCH
NCC 285
NCC 286
NCC 298
NCC 299
NCC 31
NCC 33
NCC 333
NCC 350
NCC 350 MHC
NCC 359
NCC 378
NCC 38
NCC 4
NCC 42
NCC 421
NCC 421 RCH
NCC 422
NCC 422 RCH
NCC 423
NCC 429
NCC 43 HO
NCC 43 LVH
NCC 43 MHC
NCC 43 RCH
NCC 432
NCC 44
NCC 447
NCC 474
NCC 475
NCC 484
NCC 485
NCC 486
NCC 494
NCC 499
NCC 5
NCC 5 RCH
NCC 52
NCC 526
NCC 53
NCC 535
NCC 536
NCC 57
NCC 571
NCC 574
NCC 59
NCC 6
NCC 62
NCC 64
NCC 66
NCC 70
NCC 73
NCC 77
NCC 78
NCC 80
NCC 80 RCH
NCC 82
NCC 87
NCC 9
NCC 98
Standard Charges


Claude prompt:
```
one of the top level keys is NCC 14, please select the records for only this key and print the first 10 records in it using `jq`

```

In [18]:
!jq '.["NCC 14"] | .[0:10]' /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON

[
  {
    "Header": "Lehigh Valley Hospital-Cedar Crest"
  },
  {
    "Header": "Lehigh Valley Hospital-Muhlenberg"
  },
  {
    "Header": "Lehigh Valley Hospital-Tilghman Surgery Center"
  },
  {
    "Header": "Lehigh Valley Hospital-17th Street"
  },
  {
    "Payment Type": "MS DRG",
    "Description": "HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM WITH MCC",
    "Billing Code": "MS001",
    "Payor:CAPITAL_BLUE_CROSS_Plan:CBC_KEYSTONE_HEALTH_PLAN_CENTRAL_KIDS_IP": "$245,068.11",
    "Payor:CAPITAL_BLUE_CROSS_Plan:CBC_KEYSTONE_HEALTH_PLAN_CENTRAL_KIDS_OP": "N/A"
  },
  {
    "Payment Type": "MS DRG",
    "Description": "HEART TRANSPLANT OR IMPLANT OF HEART ASSIST SYSTEM WITHOUT MCC",
    "Billing Code": "MS002",
    "Payor:CAPITAL_BLUE_CROSS_Plan:CBC_KEYSTONE_HEALTH_PLAN_CENTRAL_KIDS_IP": "$146,820.91",
    "Payor:CAPITAL_BLUE_CROSS_Plan:CBC_KEYSTONE_HEALTH_PLAN_CENTRAL_KIDS_OP": "N/A"
  },
  {
    "Payment Type": "MS DRG",
    "Description": "ECMO OR TRACHEOSTOMY WITH MV >96 HOU

Claude prompt:

> write a jq command to retrieve the records that do not have the key "Header" from the file /Users/me/data/payless_health/231689692_Lehigh_Valley_Hospital_StandardCharges.ISO-8859-1.utf8.nobackslash.JSON

next , print records with out "HEADER'" field, or split into 83 files and then skip "Header" field with JQ, then split using the colon into values, then ready for duckdb and box and whisker plot


